# Financial Sentiment Analysis
## Part 1: Dataset Selection & Research Analysis

**Dataset:** Financial PhraseBank (Malo et al., 2014)

**Objective:** Analyze the dataset, perform EDA, and review related work for financial sentiment classification.

---

## Setup and Imports

In [ ]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Project modules
from config.paths import FIGURES_DIR, REPORTS_DIR, PROCESSED_DATA_DIR
from config.params import RANDOM_SEED, LABEL_NAMES, SENTIMENT_COLORS
from src.data.loader import load_financial_phrasebank, DataLoader
from src.data.preprocessor import FinancialTextPreprocessor
from src.data.analyzer import DatasetAnalyzer, export_to_report
from src.visualization.plots import (
    set_plot_style,
    plot_label_distribution,
    plot_text_length_distribution,
    plot_word_frequency,
    generate_wordcloud,
    generate_all_wordclouds,
    plot_sentiment_scatter,
)
from src.utils.helpers import setup_logging, set_random_seed, timer

# Setup
set_random_seed(RANDOM_SEED)
set_plot_style()
logger = setup_logging()

print(f"Project root: {PROJECT_ROOT}")
print(f"Python version: {sys.version}")
print("Setup complete!")

---
# Section 1: Dataset Overview

## 1.1 Dataset Description

The **Financial PhraseBank** dataset contains sentences from financial news, annotated for sentiment polarity. It was created by Malo et al. (2014) and is widely used for financial sentiment analysis research.

### Key Characteristics:
- **Domain:** Financial news and reports
- **Language:** English
- **Task:** Sentiment classification (3 classes)
- **Annotation:** Multiple annotators with agreement thresholds

### Why Financial PhraseBank?

1. **Domain Specificity:** Specifically designed for financial text analysis
2. **Quality Annotations:** Multiple annotators with varying agreement levels
3. **Benchmark Status:** Widely used in financial NLP research
4. **Manageable Size:** Suitable for fine-tuning experiments
5. **Real-world Relevance:** Sentences from actual financial news

## 1.2 Loading the Dataset

In [ ]:
# Load dataset with 75% agreement threshold
# This provides a balance between data quality and quantity

with timer("Dataset loading"):
    df = load_financial_phrasebank(
        agreement_level="sentences_75agree",
        source="huggingface",
        save_local=True
    )

print(f"\nDataset loaded: {len(df):,} samples")
print(f"Columns: {list(df.columns)}")

In [ ]:
# Display sample data
print("\n=== Sample Data ===")
df.head(10)

In [ ]:
# Dataset info
print("\n=== Dataset Info ===")
df.info()

## 1.3 Comparing Agreement Levels

In [ ]:
# Load all agreement levels for comparison
agreement_levels = [
    "sentences_50agree",
    "sentences_66agree", 
    "sentences_75agree",
    "sentences_allagree"
]

comparison_data = []

for level in agreement_levels:
    try:
        temp_df = load_financial_phrasebank(agreement_level=level, save_local=False)
        counts = temp_df['label_name'].value_counts()
        comparison_data.append({
            'Agreement Level': level.replace('sentences_', '').replace('agree', '% agree'),
            'Total Samples': len(temp_df),
            'Positive': counts.get('positive', 0),
            'Neutral': counts.get('neutral', 0),
            'Negative': counts.get('negative', 0),
        })
    except Exception as e:
        print(f"Error loading {level}: {e}")

comparison_df = pd.DataFrame(comparison_data)
print("\n=== Agreement Level Comparison ===")
comparison_df

## 1.4 Dataset Summary Table

In [ ]:
# Create summary table for report
summary_table = pd.DataFrame({
    'Property': [
        'Dataset Name',
        'Source',
        'Domain',
        'Language',
        'Task Type',
        'Number of Classes',
        'Total Samples (75% agree)',
        'Agreement Level Used',
    ],
    'Value': [
        'Financial PhraseBank',
        'Malo et al. (2014) / HuggingFace',
        'Financial News',
        'English',
        'Multi-class Sentiment Classification',
        '3 (Positive, Neutral, Negative)',
        f'{len(df):,}',
        '75% annotator agreement',
    ]
})

print("\n=== Dataset Summary ===")
summary_table

---
# Section 2: Data Quality Analysis

## 2.1 Initialize Preprocessor and Analyzer

In [ ]:
# Initialize preprocessor
preprocessor = FinancialTextPreprocessor()

# Preprocess data
with timer("Data preprocessing"):
    df_processed = preprocessor.preprocess_dataframe(
        df,
        text_column='sentence',
        remove_duplicates=False,  # Keep duplicates for analysis
        add_text_features=True
    )

print(f"\nProcessed dataset: {len(df_processed):,} samples")
print(f"New columns: {[c for c in df_processed.columns if c not in df.columns]}")

In [ ]:
# Initialize analyzer
analyzer = DatasetAnalyzer(df_processed, text_column='sentence', label_column='label_name')

## 2.2 Missing Values Check

In [ ]:
# Check missing values
missing_stats = preprocessor.check_missing_values(df_processed)

print("\n=== Missing Values Analysis ===")
missing_stats[missing_stats['missing_count'] > 0] if missing_stats['missing_count'].sum() > 0 else print("No missing values found!")
missing_stats

## 2.3 Duplicate Analysis

In [ ]:
# Check duplicates
duplicate_texts = df_processed['sentence'].duplicated().sum()
duplicate_rows = df_processed.duplicated().sum()

print("\n=== Duplicate Analysis ===")
print(f"Duplicate texts: {duplicate_texts} ({duplicate_texts/len(df_processed)*100:.2f}%)")
print(f"Duplicate rows (all columns): {duplicate_rows} ({duplicate_rows/len(df_processed)*100:.2f}%)")

# Show examples of duplicates
if duplicate_texts > 0:
    print("\nExample duplicate sentences:")
    duplicated_sentences = df_processed[df_processed['sentence'].duplicated(keep=False)]
    duplicated_sentences.groupby('sentence').size().sort_values(ascending=False).head(5)

## 2.4 Text Statistics

In [ ]:
# Get text statistics
text_stats = analyzer.get_text_stats()

# Create text statistics table
text_stats_table = pd.DataFrame({
    'Metric': ['Minimum', 'Maximum', 'Mean', 'Median', 'Std Dev'],
    'Word Count': [
        text_stats['word_count']['min'],
        text_stats['word_count']['max'],
        f"{text_stats['word_count']['mean']:.1f}",
        f"{text_stats['word_count']['median']:.1f}",
        f"{text_stats['word_count']['std']:.1f}",
    ],
    'Character Length': [
        text_stats['character_length']['min'],
        text_stats['character_length']['max'],
        f"{text_stats['character_length']['mean']:.1f}",
        f"{text_stats['character_length']['median']:.1f}",
        f"{text_stats['character_length']['std']:.1f}",
    ],
})

print("\n=== Text Length Statistics ===")
text_stats_table

In [ ]:
# Text stats by class
print("\n=== Text Statistics by Sentiment Class ===")
stats_by_class = pd.DataFrame(text_stats['text_stats_by_class']).T
stats_by_class = stats_by_class.round(2)
stats_by_class

## 2.5 Data Quality Summary

In [ ]:
# Get comprehensive data quality report
quality_report = analyzer.check_data_quality()

quality_table = pd.DataFrame({
    'Quality Metric': [
        'Total Samples',
        'Duplicate Texts',
        'Missing Values',
        'Empty Texts',
        'Short Texts (<3 words)',
        'Very Long Texts (>99th percentile)',
        'Data Quality Score',
    ],
    'Value': [
        f"{len(df_processed):,}",
        f"{quality_report['duplicate_texts']} ({quality_report['duplicate_texts']/len(df_processed)*100:.1f}%)",
        str(quality_report['total_missing']),
        str(quality_report['empty_texts']),
        str(quality_report['short_texts_under_3_words']),
        str(quality_report['long_texts_above_99th_percentile']),
        f"{quality_report['data_quality_score']:.1f}%",
    ],
    'Status': [
        '✓',
        '⚠' if quality_report['duplicate_texts'] > 0 else '✓',
        '✓' if quality_report['total_missing'] == 0 else '✗',
        '✓' if quality_report['empty_texts'] == 0 else '✗',
        '✓',
        '✓',
        '✓' if quality_report['data_quality_score'] >= 95 else '⚠',
    ]
})

print("\n=== Data Quality Summary ===")
quality_table

---
# Section 3: Exploratory Data Analysis

## 3.1 Label Distribution (Graph 1)

In [ ]:
# Get label distribution
label_dist = analyzer.get_label_distribution()
print("\n=== Label Distribution ===")
label_dist

In [ ]:
# Plot label distribution
fig, ax = plot_label_distribution(
    df_processed,
    label_column='label_name',
    title='Financial PhraseBank - Sentiment Label Distribution',
    save=True
)
plt.show()

In [ ]:
# Calculate class imbalance metrics
basic_stats = analyzer.get_basic_stats()

print("\n=== Class Imbalance Analysis ===")
print(f"Imbalance Ratio (max/min): {basic_stats['imbalance_ratio']:.2f}")
print(f"\nClass proportions:")
for label, prop in basic_stats['class_balance'].items():
    print(f"  {label}: {prop*100:.1f}%")

## 3.2 Text Length Distribution (Graph 2)

In [ ]:
# Plot text length distribution
fig, axes = plot_text_length_distribution(
    df_processed,
    length_column='word_count',
    label_column='label_name',
    title='Text Length Distribution by Sentiment',
    save=True
)
plt.show()

In [ ]:
# Additional: Distribution statistics
print("\n=== Word Count Percentiles ===")
percentiles = [10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    value = df_processed['word_count'].quantile(p/100)
    print(f"  {p}th percentile: {value:.0f} words")

## 3.3 Word Frequency Analysis (Graph 3)

In [ ]:
# Get word frequencies by sentiment
word_freq_df = preprocessor.get_word_frequencies(
    df_processed,
    text_column='sentence_clean',
    top_n=20,
    by_label=True,
    label_column='label_name'
)

print("\n=== Top 10 Words by Sentiment ===")
for label in ['positive', 'neutral', 'negative']:
    print(f"\n{label.upper()}:")
    top_words = word_freq_df[word_freq_df['label'] == label].head(10)
    for _, row in top_words.iterrows():
        print(f"  {row['word']}: {row['count']}")

In [ ]:
# Plot word frequency
fig, axes = plot_word_frequency(
    word_freq_df,
    title='Top 15 Most Frequent Words by Sentiment',
    top_n=15,
    save=True
)
plt.show()

## 3.4 Word Clouds (Graph 4)

In [ ]:
# Generate word clouds for each sentiment class
wordcloud_figs = generate_all_wordclouds(
    df_processed,
    text_column='sentence_clean',
    label_column='label_name',
    save=True,
    stopwords=preprocessor.stopwords
)

# Display word clouds
for label, (fig, ax) in wordcloud_figs.items():
    plt.figure(fig.number)
    plt.show()

## 3.5 Sentiment vs Text Length (Graph 5)

In [ ]:
# Scatter plot: Text length vs sentiment
fig, ax = plot_sentiment_scatter(
    df_processed,
    x_column='word_count',
    label_column='label_name',
    title='Text Length vs Sentiment Distribution',
    sample_size=2000,
    save=True
)
plt.show()

In [ ]:
# Statistical test: Is text length significantly different across sentiments?
from scipy import stats

groups = [df_processed[df_processed['label_name'] == label]['word_count'] 
          for label in ['negative', 'neutral', 'positive']]

# Kruskal-Wallis H-test (non-parametric)
h_stat, p_value = stats.kruskal(*groups)

print("\n=== Statistical Test: Text Length by Sentiment ===")
print(f"Kruskal-Wallis H-statistic: {h_stat:.4f}")
print(f"P-value: {p_value:.4e}")
print(f"Conclusion: {'Significant difference' if p_value < 0.05 else 'No significant difference'} in text length across sentiments")

## 3.6 Sample Sentences by Sentiment

In [ ]:
# Display sample sentences for each sentiment
print("\n=== Sample Sentences by Sentiment ===")

for sentiment in ['positive', 'neutral', 'negative']:
    print(f"\n{'='*60}")
    print(f"  {sentiment.upper()} EXAMPLES")
    print(f"{'='*60}")
    
    samples = df_processed[df_processed['label_name'] == sentiment].sample(n=3, random_state=42)
    for i, (_, row) in enumerate(samples.iterrows(), 1):
        print(f"\n{i}. \"{row['sentence']}\"")
        print(f"   [Words: {row['word_count']}, Chars: {row['char_count']}]")

---
# Section 4: Related Work

## 4.1 Literature Review

This section reviews key papers in financial sentiment analysis that are relevant to our task.

### Paper 1: FinBERT (Araci, 2019)

**Title:** "FinBERT: Financial Sentiment Analysis with Pre-trained Language Models"

**Authors:** Dogu Araci

**Year:** 2019

**Key Contributions:**
- Pre-trained BERT model on financial corpus (TRC2-financial)
- Domain-specific language model for financial NLP
- State-of-the-art results on Financial PhraseBank

**Dataset:** Financial PhraseBank (sentences_allagree)

**Method:** Fine-tuned BERT pre-trained on financial texts

**Results:**
- Accuracy: 97.2%
- F1-score: 0.88 (macro)

**Relevance:** Direct baseline for our task; demonstrates effectiveness of domain-specific pre-training

### Paper 2: Good Debt or Bad Debt (Malo et al., 2014)

**Title:** "Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts"

**Authors:** Pekka Malo, Ankur Sinha, Pekka Korhonen, Jyrki Wallenius, Pyry Takala

**Year:** 2014

**Key Contributions:**
- Created the Financial PhraseBank dataset
- Analyzed annotation agreement patterns
- Established benchmark for financial sentiment

**Dataset:** Financial PhraseBank (created in this paper)

**Method:** SVM with n-grams and lexicon features

**Results:**
- Accuracy: 72-77% depending on agreement level

**Relevance:** Original dataset paper; provides annotation guidelines and baseline results

### Paper 3: Common Mistakes and Silver Bullets (Theil et al., 2018)

**Title:** "Financial Sentiment Analysis: An Investigation into Common Mistakes and Silver Bullets"

**Authors:** Christoph K. Theil, Samuel Broscheit, Heiner Stuckenschmidt

**Year:** 2018

**Key Contributions:**
- Identified common pitfalls in financial sentiment analysis
- Analyzed dataset biases and evaluation issues
- Provided best practices for reproducible research

**Dataset:** Multiple (including Financial PhraseBank)

**Method:** Various ML models (SVM, CNN, LSTM)

**Key Findings:**
- Data leakage is common in existing studies
- Class imbalance significantly affects results
- Proper cross-validation is crucial

**Relevance:** Provides methodological guidance; warns about evaluation pitfalls

## 4.2 Comparison Table

In [ ]:
# Create related work comparison table
related_work_df = pd.DataFrame({
    'Paper': [
        'FinBERT (Araci, 2019)',
        'Malo et al. (2014)',
        'Theil et al. (2018)',
        'This Project (Proposed)'
    ],
    'Dataset': [
        'Financial PhraseBank (allagree)',
        'Financial PhraseBank',
        'Multiple datasets',
        'Financial PhraseBank (75agree)'
    ],
    'Method': [
        'Fine-tuned BERT (domain-specific)',
        'SVM + n-grams + lexicon',
        'SVM, CNN, LSTM comparison',
        'Fine-tuned RoBERTa/FinBERT'
    ],
    'Accuracy': ['97.2%', '72-77%', '~80%', 'TBD'],
    'F1 (Macro)': ['0.88', 'N/A', '~0.70', 'Target: 0.85+']
})

print("\n=== Related Work Comparison ===")
related_work_df

## 4.3 Key Insights from Literature

1. **Domain Pre-training Matters:** FinBERT's success demonstrates that pre-training on financial texts significantly improves performance compared to general-purpose models.

2. **Class Imbalance:** The neutral class dominates (~53%), which requires careful handling during training (weighted loss, oversampling, etc.).

3. **Agreement Level Trade-off:** Higher agreement levels have more reliable labels but fewer samples. 75% agreement offers a good balance.

4. **Evaluation Best Practices:**
   - Use stratified splits
   - Report macro F1 (handles imbalance)
   - Use proper cross-validation
   - Report per-class metrics

5. **Baseline Expectations:** 
   - Traditional ML: 70-80% accuracy
   - Fine-tuned transformers: 85-95% accuracy
   - Domain-specific transformers: 95%+ accuracy

---
# Section 5: Problem Statement & Hypothesis

## 5.1 Real-World Problem

### Business Context

Financial analysts and traders need to process thousands of news articles, press releases, and social media posts daily. Manual analysis is:
- **Time-consuming:** Hours to read and analyze
- **Inconsistent:** Human bias and fatigue
- **Not scalable:** Cannot handle real-time data volume

### Problem Statement

> **Develop an automated sentiment classification system that accurately classifies financial news sentences into positive, negative, or neutral sentiment to support investment decision-making.**

### Applications

1. **Algorithmic Trading:** Incorporate sentiment signals into trading strategies
2. **Risk Management:** Early detection of negative sentiment around holdings
3. **Portfolio Analysis:** Monitor sentiment trends for portfolio companies
4. **Market Research:** Aggregate sentiment for sector/industry analysis

## 5.2 Research Hypothesis

### Primary Hypothesis

> **H1:** A fine-tuned transformer model (RoBERTa or FinBERT) will achieve **macro F1 score > 0.85** on the Financial PhraseBank sentiment classification task.

### Secondary Hypotheses

> **H2:** Domain-specific pre-training (FinBERT) will outperform general-purpose models (RoBERTa-base) by at least 3% in macro F1.

> **H3:** Fine-tuning approach will outperform zero-shot/few-shot prompting with larger LLMs on this domain-specific task.

### Rationale

- Literature shows FinBERT achieving ~88% F1, suggesting 85%+ is achievable
- Financial text has domain-specific terminology that benefits from specialized models
- Small, focused datasets favor fine-tuning over prompting

## 5.3 Approach Justification

### Fine-Tuning vs. Prompting

| Aspect | Fine-Tuning | Prompting (LLM) |
|--------|-------------|------------------|
| **Latency** | Fast inference (ms) | Slow (API calls, seconds) |
| **Cost** | One-time training | Per-token API costs |
| **Customization** | Full control | Limited by prompts |
| **Domain Knowledge** | Learns from data | Relies on pre-training |
| **Reproducibility** | Deterministic | Non-deterministic |
| **Deployment** | Self-hosted | API dependency |

**Decision:** Fine-tuning is preferred for:
- Production deployment requirements (latency, cost)
- Domain-specific vocabulary
- Reproducible results

### Model Selection

| Model | Pros | Cons |
|-------|------|------|
| **RoBERTa-base** | Strong baseline, well-documented | No domain knowledge |
| **FinBERT** | Domain pre-trained, SOTA | Less community support |
| **DistilBERT** | Fast, efficient | Lower capacity |

**Plan:** Compare RoBERTa-base vs. FinBERT to validate H2

## 5.4 Expected Outcomes

### Deliverables (Part 2)

1. **Trained Model:** Fine-tuned sentiment classifier
2. **Evaluation Report:** Comprehensive metrics (accuracy, F1, confusion matrix)
3. **Error Analysis:** Common failure cases and patterns
4. **Model Comparison:** RoBERTa vs. FinBERT performance

### Success Criteria

| Metric | Target | Stretch Goal |
|--------|--------|-------------|
| Macro F1 | > 0.85 | > 0.90 |
| Accuracy | > 85% | > 90% |
| Negative Class F1 | > 0.75 | > 0.80 |

### Integration Pipeline

```
News Feed → Sentence Extraction → Sentiment Model → Aggregation → Trading Signal
```

## 5.5 Project Timeline

| Phase | Description | Status |
|-------|-------------|--------|
| Part 1 | Dataset Selection & Analysis | ✓ Complete |
| Part 2 | Model Training & Evaluation | Upcoming |
| Part 3 | Analysis & Report | Upcoming |

---
# Section 6: Save Results and Generate Report

In [ ]:
# Save processed data
processed_path = preprocessor.save_processed_data(
    df_processed,
    filename="financial_phrasebank_processed.csv"
)
print(f"Processed data saved to: {processed_path}")

In [ ]:
# Save statistics report
stats_path = analyzer.save_stats_report(format='csv')
print(f"Statistics saved to: {stats_path}")

In [ ]:
# Generate markdown report
report_path = export_to_report(analyzer)
print(f"Report generated: {report_path}")

In [ ]:
# List all generated files
print("\n=== Generated Output Files ===")

print("\nFigures:")
for fig_file in FIGURES_DIR.glob("*.png"):
    print(f"  - {fig_file.name}")

print("\nReports:")
for report_file in REPORTS_DIR.glob("*"):
    print(f"  - {report_file.name}")

print("\nProcessed Data:")
for data_file in PROCESSED_DATA_DIR.glob("*.csv"):
    print(f"  - {data_file.name}")

---
# Summary

## Key Findings from Part 1

1. **Dataset:** Financial PhraseBank with ~3,453 samples (75% agreement)
2. **Class Distribution:** Imbalanced - Neutral (53%), Positive (32%), Negative (15%)
3. **Text Characteristics:** Average ~23 words per sentence
4. **Data Quality:** High quality with minimal issues

## Next Steps (Part 2)

1. Split data into train/validation/test sets
2. Implement and train RoBERTa-base classifier
3. Implement and train FinBERT classifier
4. Compare models and evaluate hypotheses
5. Perform error analysis

---
*End of Part 1: Dataset Selection & Research Analysis*